<a href="https://colab.research.google.com/github/NiXxXO-DK/DS/blob/master/m3_a1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download and install tonsorflow and keras
#!pip download -qq keras
#!pip install -qq tensorflow
#!pip install keras

In [2]:
#import packages
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [3]:
#Import tweet data
header = ['text', 'sentiment']
data = pd.read_csv('https://github.com/SDS-AAU/SDS-master/raw/master/M3/assignments/find_elon.gz',compression='gzip', names=header)
pd.DataFrame(data).sort_index(ascending=True)

,text,sentiment
0,0,1
1,@WhatsupFranks @lexfridman NPCs all the way up,1
2,We’ll only do one or two at a time. @S_Padival...,0
3,@Erdayastronaut @SpaceX SN4 with F9 engine is ...,0
4,@LarryKellogg @arctechinc The Tesla FSD comput...,1
...,...,...
3150,"In fact, the tunnel will not be there for seve...",0
3151,Tesla dual motor means there is a motor in fro...,1
3152,"Hey babe, fancy a hot date …? 😉 @4thFromOurStar",1
3153,"If you buy a car, you will get a self-driving ...",0


In [4]:
#Change the sentiment to true/false
data['sentiment'] = data['sentiment'].map({0: "False", 1: "True"})
print(data)

                                                   text sentiment
0                                                     0      True
1        @WhatsupFranks @lexfridman NPCs all the way up      True
2     We’ll only do one or two at a time. @S_Padival...     False
3     @Erdayastronaut @SpaceX SN4 with F9 engine is ...     False
4     @LarryKellogg @arctechinc The Tesla FSD comput...      True
...                                                 ...       ...
3150  In fact, the tunnel will not be there for seve...     False
3151  Tesla dual motor means there is a motor in fro...      True
3152    Hey babe, fancy a hot date …? 😉 @4thFromOurStar      True
3153  If you buy a car, you will get a self-driving ...     False
3154                    @richardreinariv He has no clue      True

[3155 rows x 2 columns]


In [5]:
#Clean and Tokenize the data
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == "True"].size)
print(data[ data['sentiment'] == "False"].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

3156
3154


In [6]:
#Create  the model with an embedding layer as input

embed_dim = 256
lstm_out = 512

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 85, 256)           2560000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 85, 256)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 512)               1574912   
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 4,135,938
Trainable params: 4,135,938
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
#Splitting the data to training and test set
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2524, 85) (2524, 2)
(631, 85) (631, 2)


In [12]:
#tain the model
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
79/79 - 38s - loss: 0.0693 - accuracy: 0.9778 - 38s/epoch - 485ms/step
Epoch 2/5
79/79 - 38s - loss: 0.0189 - accuracy: 0.9952 - 38s/epoch - 485ms/step
Epoch 3/5
79/79 - 39s - loss: 0.0056 - accuracy: 0.9988 - 39s/epoch - 490ms/step
Epoch 4/5
79/79 - 38s - loss: 0.0029 - accuracy: 0.9996 - 38s/epoch - 485ms/step
Epoch 5/5
79/79 - 38s - loss: 0.0172 - accuracy: 0.9941 - 38s/epoch - 485ms/step


In [13]:
#test how accuract the model is on the test data 
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

20/20 - 2s - loss: 0.6020 - accuracy: 0.8257 - 2s/epoch - 84ms/step
score: 0.60
acc: 0.83


In [ ]:
!jupyter nbconvert "/content/m3_a1.ipynb"